In [49]:
import numpy as np

from processes.reserves import light_weight_reserve

intensities = [{"warm_up_intensity": 1, "main_intensity": 10, "is_main": True},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False},
               {"warm_up_intensity": 1, "main_intensity": 10, "is_main": False}]
light_weight_reserve(intensities)

0.16995460180153946

$$\large{Task^1}$$\
$$\large{\lambda = (1, 1, 1, 1, 1)}$$\
$$\large{\Lambda = (10, 10, 10, 10, 10)}$$\
$$\large{main = 1}$$

In [50]:
def lln(n):
    return np.mean(np.vectorize(lambda _: light_weight_reserve(intensities))(np.arange(1, n + 1)))


experiments_lln = np.vectorize(lambda n: lln(n))(np.arange(1, 301))
experiments_lln[:20]

array([0.26094685, 0.50164408, 0.3037126 , 0.41476059, 0.43520929,
       0.3923836 , 0.47472621, 0.4355808 , 0.35153981, 0.46331582,
       0.40477233, 0.60598351, 0.46516202, 0.46162239, 0.40850746,
       0.41581807, 0.45416023, 0.40660412, 0.33392554, 0.40865722])

$$\mathbb{E}\mathit{T_n} = \sum_{k=1}^{n}\frac{1}{\Lambda + (k-1)\lambda}$$

In [51]:
real_mean = np.sum(np.fromfunction(lambda i: 1 / (10 + i), (5,)))
print(real_mean)

0.42259407259407256


In [52]:
import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'

import plotly.express as px
N = np.arange(1, 301)
fig = px.scatter(x=N, y=experiments_lln)

fig.update_layout(
    title=r"$\large{\text{Law of large numbers}}$",
    xaxis_title=r"$\Large{n}$",
    yaxis_title=r"$\Large{\frac{S_n}{n}}$",
)
fig.add_hline(y=real_mean, line_color="pink")


In [53]:

fig = px.scatter(x=N, y=experiments_lln, animation_frame=N, range_y=[0, 0.8],range_x=[1, 300])
fig.update_layout(
    title=r"$\large{\text{Law of large numbers}}$",
    xaxis_title=r"$\Large{n}$",
    yaxis_title=r"$\Large{\frac{S_n}{n}}$",
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.05
    )
)
sliders = [dict(
    currentvalue=dict(prefix = r'n = ')
)]

fig.update_layout(
    sliders=sliders
)
fig.add_hline(y=real_mean, line_color="pink")



$$\mathbb{D}\mathit{T_n} = \mathbb{D}\sum_{k=1}^{n}\mathit{E}_{\Lambda + (k - 1)\lambda} = \sum_{k=1}^{n}\mathbb{D}\mathit{E}_{\Lambda + (k - 1)\lambda} = \sum_{k=1}^{n}\frac{1}{(\Lambda + (k - 1)\lambda)^2}$$

In [54]:
real_variance = np.sum(np.fromfunction(lambda i: 1 / (10 + i) ** 2, (5,)))
print(real_variance)

0.03622810783400194


In [55]:
def clt(n):
    return np.sqrt(n) * (lln(n) - real_mean) / np.sqrt(real_variance)

In [56]:
experiments_clt = np.vectorize(lambda n: clt(n))(np.full((1000, ), 50))
experiments_clt.size

1000

In [57]:
px.histogram(experiments_clt)

In [58]:
import scipy.stats as sps

norm_experiments = sps.norm.rvs(size=1000)
px.histogram(norm_experiments, )


In [59]:
params_of_experiment = np.mgrid[10:110:10, 1:1001][0]

In [60]:
clt_experiments = np.vectorize(lambda n: clt(n))(params_of_experiment)

In [61]:
clt_experiments_n = clt_experiments[:,:1000]
norm_experiments_n = norm_experiments[:1000]

In [62]:
clt_experiments_n[0].size

1000

In [63]:
norm_experiments_n.size

1000

In [64]:
import plotly.graph_objects as go

def get_histogram(data, left, right, count_bins):
    bins, step = np.linspace(left, right, count_bins + 2, retstep=True)
    bins_points = .5 * (bins[:-1] + bins[1:])
    density, bins = np.histogram(data, bins=bins, density=True)
    density /= density.sum()
    return bins_points, density


$$\text{$A_j$ is a j bin}$$\
$$v_j = \sum_{i=0}^{1000}\mathit{I}(clt_i \in A_j)$$\
$$u_j = \sum_{i=0}^{1000}\mathit{I}(norm_i \in A_j)$$\
$$\text{Distance between distributions}$$\
$$\text{D} = \sum_{j=0}^{39}(v_j - u_j)^2$$

In [65]:
_, norm_density = get_histogram(norm_experiments_n, -4, 4, 39)
np.sum((get_histogram(clt_experiments_n[0], -4, 4, 39)[1] - norm_density)**2) 

0.002367999999999997

In [66]:
import copy

layout = dict(
    title=r"$\text{Central limit theorem visualization}$",
    barmode="overlay",
    hovermode="closest",
    updatemenus=[dict(
    type="buttons",
    buttons=[dict(label="Play", method="animate",
                  args=[None, 
                        {"frame": {"duration": 1000, "redraw": True},
                         "transition": {"duration": 300, "easing": "quadratic-in-out"}
                        }],
                 ),
             dict(label="Pause", method="animate",
                  args=[None,
                           {"frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0}}],
                         )],
    )],
    annotations = [dict(x=2.2, y=0.075,
                        text=repr("$Distance = " + "{:.5f}".format(np.sum(
                            (get_histogram(clt_experiments_n[0], -4, 4, 39)[1] - norm_density)**2
                        )) + "$"),
                        showarrow=False,
                        font=dict(size=14)),
                  ]
)

frames = list()

for i in range(len(clt_experiments)):
    frame_layout = copy.deepcopy(layout)
    layout["annotations"] = [dict(x=2.2, y=0.075,
                                  text=repr("$Distance = " + "{:.5f}".format(np.sum(
                                      (get_histogram(clt_experiments_n[i], -4, 4, 39)[1] - norm_density)**2
                                  )) + "$"),
                                  showarrow=False,
                                  font=dict(size=14)),
                            ]
    frames.append(go.Frame(
        data = [
        go.Histogram(name=r"$\text{Standard normal distribution}$", 
                     x=norm_experiments_n, xbins=go.histogram.XBins(size=.2), 
                     histnorm="probability"),
        go.Histogram(name=repr("$\text{CLT with } n = "+ str((i + 1) * 10) +"$"), 
                     x=clt_experiments_n[i], xbins=go.histogram.XBins(size=.2), 
                     histnorm="probability"),
        ],
        layout=frame_layout
    ))
    

fig = go.Figure(
    data = [
        go.Histogram(name=r"$\text{Standard normal distribution}$", x=norm_experiments_n, xbins=go.histogram.XBins(size=.2), histnorm="probability"),
        go.Histogram(name=repr("$\text{CLT with } n = 10$"), x=clt_experiments_n[0], xbins=go.histogram.XBins(size=.2), histnorm="probability")
    ],
    layout=go.Layout(layout),
    frames=frames,
)
fig.update_traces(opacity=0.75)
fig.show()

$$\large{Task^2}$$\
$$\large{\lambda = (1, 2, 2, 4)}$$\
$$\large{\Lambda = (5, 5, 7, 11)}$$\
$$\large{main = ?}$$

In [67]:
import copy


In [68]:
from itertools import permutations


intensities = [
    {"warm_up_intensity": 1, "main_intensity": 5, "is_main": False},
    {"warm_up_intensity": 2, "main_intensity": 5, "is_main": False},
    {"warm_up_intensity": 2, "main_intensity": 7, "is_main": False},
    {"warm_up_intensity": 4, "main_intensity": 11, "is_main": False},
]

def initialize_system(system):
    system = copy.deepcopy(system)
    system[0]["is_main"] = True
    return system

systems = np.array([initialize_system(system) for system in list(permutations(intensities, 4))])
systems

array([[{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity': 5, 'is_main': False},
        {'warm_up_intensity': 2, 'main_intensity': 7, 'is_main': False},
        {'warm_up_intensity': 4, 'main_intensity': 11, 'is_main': False}],
       [{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity': 5, 'is_main': False},
        {'warm_up_intensity': 4, 'main_intensity': 11, 'is_main': False},
        {'warm_up_intensity': 2, 'main_intensity': 7, 'is_main': False}],
       [{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity': 7, 'is_main': False},
        {'warm_up_intensity': 2, 'main_intensity': 5, 'is_main': False},
        {'warm_up_intensity': 4, 'main_intensity': 11, 'is_main': False}],
       [{'warm_up_intensity': 1, 'main_intensity': 5, 'is_main': True},
        {'warm_up_intensity': 2, 'main_intensity'

In [69]:
def get_experiments_system(system, n):
    return np.vectorize(lambda _: light_weight_reserve(system))(np.empty(n))

def get_experiments_systems(systems, n):
    return np.array([get_experiments_system(system, n) for system in systems])

experiments_systems = get_experiments_systems(systems, 1000)
experiments_systems

array([[0.39366727, 0.26307552, 0.21627696, ..., 0.18688552, 0.20097191,
        0.30060717],
       [0.41413836, 0.30350028, 0.59946849, ..., 0.47584418, 0.14321217,
        0.45008386],
       [0.54351745, 0.29875754, 0.70904607, ..., 0.49350278, 0.7767223 ,
        0.48441169],
       ...,
       [0.27115818, 0.32261303, 0.62462035, ..., 0.39949672, 0.33868257,
        0.20038158],
       [0.71743076, 0.33131438, 1.11005978, ..., 0.23470668, 0.36721263,
        0.35676325],
       [0.42859885, 0.5847194 , 0.61261541, ..., 0.81142317, 0.32061568,
        0.75395545]])

In [70]:
def get_means_of_experiments(experiments, axis=1):
    return np.mean(experiments, axis=axis)


sample_means = get_means_of_experiments(experiments_systems)
descriptions_systems = [repr("$"+ ','.join(["(" + str(device["warm_up_intensity"]) + ',' + str(device["main_intensity"]) + ")" for device in system]) + "$") 
                        for system in systems]

px.scatter(x=descriptions_systems, y=sample_means, labels={'x': r'$\text{systems}$', 'y': r'$\text{sample means}$'})

In [71]:
theoretical_averages = np.array([get_means_of_experiments(get_experiments_systems(systems, 100)) for _ in range(100)])

In [72]:
averages = theoretical_averages


In [73]:
theoretical_means = np.mean(theoretical_averages.T, axis=1) 

In [74]:
px.scatter(x=descriptions_systems, y=theoretical_means, labels={'x': r'$\text{systems}$', 'y': r'$\text{theoretical means}$'})

In [75]:
px.ecdf(x=experiments_systems[21])